# LVV-T2239 (1.0) ComCam Startup Procedure

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lsst.sitcom import vandv

exec_info = vandv.ExecutionInfo()
print(exec_info)


Executed by b1quint on 2022-06-15T19:51:40.647.
  Running in yagan07 at summit



In [3]:
test_execution = "LVV-E1264"

In [5]:
import asyncio
import os
import yaml

import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from astropy import time 
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
from datetime import datetime, timedelta

from lsst_efd_client import EfdClient
from lsst.ts import utils, salobj
from lsst.ts.observatory.control.maintel import ComCam

import lsst.sitcom.vandv as vandv

In [6]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [7]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [8]:
domain = salobj.Domain()

In [9]:
comcam = ComCam(domain=domain, log=log)

In [10]:
comcam.set_rem_loglevel(40)

In [11]:
await comcam.start_task

[None, None, None]

In [12]:
await comcam.standby()

In [36]:
def print_detailed_state():
    
    ccs_command_state = comcam.rem.cccamera.evt_ccsCommandState.get().substate
    detailed_state = comcam.rem.cccamera.evt_calibrationDetailedState.get().substate
    
    t = time.Time(utils.current_tai(), format="unix", scale="tai")
    t.format = "isot"
    
    print(
        f"\nComCam State: {ccs_command_state}"
        f"\nDetailed State: {detailed_state}"
        f"\nat: {t}"
    )

---
While the SummaryState is in the OFFLINE state, send an enterControl command.

The SummaryState transitions into the STANDBY state.

In [13]:
# ComCam was in the ENABLED state and we sent it to the STANDBY state.
# Need to know where/how transition from OFFLINE to STANDBY

---
While the SummaryState is in the STANDBY state, send a start command.

In [17]:
s = await comcam.get_state("cccamera")
print(s)

# We will use the higher level command `await comcam.enable()` which 
# performs all the state transitions from STANDBY to DISABLED to ENABLED.
# So we are not doing anything on this step.

State.STANDBY


---
While the SummaryState is in the DISABLED state, send an enable command.

In [18]:
await comcam.enable()

## CalibrationDetailedState

Check the current state of the CalibrationDetailedState.

In [44]:
print( comcam.rem.cccamera.evt_calibrationDetailedState.get() )

private_revCode: 5e0e464f, private_sndStamp: 1655323860.1062264, private_rcvStamp: 1655323860.1063983, private_seqNum: 1, private_identity: CCCamera, private_origin: 86409167, substate: 2, timestampTransition: 0.0, priority: 1


In [45]:
assert salobj.State.DISABLED.value == comcam.rem.cccamera.evt_calibrationDetailedState.get().substate

AssertionError: 

---
While the CalibrationDetailedState is reported to be in DISABLED state and Idle, send an enableCalibration command.

In [46]:
await comcam.rem.cccamera.cmd_enableCalibration.set_start()

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=587915846, ack=<SalRetCode.CMD_NOPERM: -300>, error=0, result='Ack : NO Command rejected: Invalid calibration state State{CalibrationState = ENABLED}')

In [47]:
assert salobj.State.ENABLED.value == comcam.rem.cccamera.evt_calibrationDetailedState.get().substate

---
While the CalibrationDetailedState is in the ENABLED state, send a startImage command.

The CalibrationDetailedState transitions into the INTEGRATING state and the  CCCamera_logevent_CalibrationDetailedState publishes INTEGRATING.

In [48]:
await comcam.rem.cccamera.cmd_startImage.set_start(
    shutter=False,  # We leave the shutter closed to avoid damaging the ccds in case of long exposure
)
print_detailed_state()
await asyncio.sleep(1)

print_detailed_state()
await comcam.rem.cccamera.cmd_discardRows.set_start(nRows=2)
await asyncio.sleep(1)

print_detailed_state()
await comcam.rem.cccamera.cmd_discardRows.set_start(nRows=5)
await asyncio.sleep(1)

print_detailed_state()
await comcam.rem.cccamera.cmd_stop.set_start()
await asyncio.sleep(1)

print_detailed_state()
await comcam.rem.cccamera.cmd_endImage.set_start()
await asyncio.sleep(1)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1876107798, ack=<SalRetCode.CMD_NOPERM: -300>, error=0, result='Ack : NO Command rejected: Invalid argument')

---
While in the INTEGRATING state and the SummaryState is ENABLED, send a discardRows command of 2 rows

In [ ]:
print_detailed_state()
# await comcam.rem.cccamera.cmd_discardRows.set_start(nRows=2)

---
While in the INTEGRATING state and the SummaryState is ENABLED, send a discardRows command of 5 rows.

In [33]:
print_detailed_state()
# await comcam.rem.cccamera.cmd_discardRows.set_start(nRows=5)

2022-06-15T20:42:54.056 

private_revCode: 45f4d9ea, private_sndStamp: 1655325064.191647, private_rcvStamp: 1655325064.1918113, private_seqNum: 966, private_identity: CCCamera, private_origin: 86409167, substate: 1, timestampTransition: 0.0, priority: 1 

private_revCode: 5e0e464f, private_sndStamp: 1655323860.1062264, private_rcvStamp: 1655323860.1063983, private_seqNum: 1, private_identity: CCCamera, private_origin: 86409167, substate: 2, timestampTransition: 0.0, priority: 1 



---
While in the INTEGRATING state and the ccsCommandState is BUSY, send a stop command.

In [35]:
print_detailed_state()
# await comcam.rem.cccamera.cmd_stop.set_start()

2022-06-15T20:43:13.941 

private_revCode: 45f4d9ea, private_sndStamp: 1655325064.191647, private_rcvStamp: 1655325064.1918113, private_seqNum: 966, private_identity: CCCamera, private_origin: 86409167, substate: 1, timestampTransition: 0.0, priority: 1 

private_revCode: 5e0e464f, private_sndStamp: 1655323860.1062264, private_rcvStamp: 1655323860.1063983, private_seqNum: 1, private_identity: CCCamera, private_origin: 86409167, substate: 2, timestampTransition: 0.0, priority: 1 



---
While in the INTEGRATING state and the ccsCommandState is IDLE, send an endImage command.

In [38]:
print_detailed_state()
# await comcam.rem.cccamera.cmd_endImage.set_start()


ComCam State: 1
Detailed State: 2
at: 2022-06-15T20:47:40.319


---
While in the ENABLED state, send a disableCalibration command.

In [40]:
print_detailed_state()
# await comcam.rem.cccamera.cmd_disableCalibration.set_start()


ComCam State: 1
Detailed State: 2
at: 2022-06-15T21:03:57.247


---
With the ShutterDetailedState in the CLOSED state, send a startImage command to open the camera shutter.

- The Camera publishes a startShutterOpen event as the camera shutter starts to open
- The ShutterDetailedState transitions from CLOSED to OPENING
- The ImageReadinessDetailedState is in the NOT_READY state

In [41]:
print_detailed_state()
# await comcam.rem.cccamera.cmd_startImage.set_start()


ComCam State: 1
Detailed State: 2
at: 2022-06-15T21:07:33.365


---
With the ShutterDetailedState in the OPENING state, wait for the camera shutter to fully open.

- The Camera publishes a endShutterOpen event when the camera shutter is open
- The ShutterDetailedState transitions from OPENING to OPEN
- The ImageReadinessDetailedState is in the GETTING_READY state while the camera shutter is opening but transitions into the READY state when it's done

---
With the ShutterDetailedState in the OPEN state, send a command to close the camera shutter.

- The Camera publishes a startShutterClose event as the camera shutter starts to shut
- The ShutterDetailedState transitions from OPEN to CLOSING

---
With the ShutterDetailedState in the CLOSING state, wait for the camera shutter to fully close.

- The Camera publishes a endShutterClose event when the camera shutter is closed
- The ShutterDetailedState transitions from CLOSING to CLOSED

---
RaftsDetailedState
- Needs_Clear
- Clearing
- Integrating
- Reading_Out
- Quiescent

In [49]:
await comcam.standby()